In [ ]:
#Please read the README txt file for dependency versions
#Also this is trained on Indian dataset so it will have bias but this is fine due to our usecase targeted that
#The whole workflow from Dataset creation to training is explained in the readme file so please read it

In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
import os

In [2]:

from matplotlib import pyplot as plt

In [3]:
tf.compat.v1.enable_eager_execution()

In [4]:
def preprocessing(x):
    layer_norm=tf.keras.layers.LayerNormalization()
    x=layer_norm(x)
    layer_bright=tf.keras.layers.RandomBrightness(factor=0.2)
    layer_contrast=tf.keras.layers.RandomContrast(factor=0.2)
    x=layer_bright(x)
    x=layer_contrast(x)
    layer_resize=tf.keras.layers.Resizing(height=224,width=224)
    x=layer_resize(x)
    return x

In [5]:
base_model=tf.keras.applications.ResNet50(
      include_top=False
  )
base_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(),
        tf.keras.metrics.FalseNegatives(),
    ],
)

In [6]:
#maybe
import tensorflow as tf

class ChannelAttention(tf.keras.layers.Layer):
    def __init__(self, in_channels, reduction_ratio=16):
        super(ChannelAttention, self).__init__()
        self.in_channels = in_channels
        self.reduction_ratio = reduction_ratio

        self.avg_pool = tf.keras.layers.AveragePooling2D(pool_size=(7, 7),strides=(1, 1), padding='same')
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size=(7, 7),strides=(1, 1), padding='same')

        self.fc1 = tf.keras.layers.Dense(in_channels // reduction_ratio, activation='relu', use_bias=False)
        self.fc2 = tf.keras.layers.Dense(in_channels, use_bias=False)

        self.sigmoid = tf.keras.layers.Activation('sigmoid')

    def call(self, inputs):
        avg_out = self.avg_pool(inputs)
        max_out = self.max_pool(inputs)

        avg_out = self.fc2(self.fc1(avg_out))
        max_out = self.fc2(self.fc1(max_out))

        out = avg_out + max_out
        out = self.sigmoid(out)

        return out

channel_attent = ChannelAttention(in_channels=2048)
input_tensor = tf.random.normal((1, 7, 7, 2048))  
output_tensor = channel_attent(input_tensor)
print(output_tensor.shape) 


(1, 7, 7, 2048)


In [7]:
import tensorflow as tf

class SpatialAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters=1, kernel_size=7, strides=1, padding='same', activation='sigmoid')

    def call(self, inputs):
        # Average pooling across channels
        avg_pool = tf.reduce_mean(inputs, axis=3, keepdims=True)
        max_pool = tf.reduce_max(inputs, axis=3, keepdims=True)
        concatenated = tf.concat([avg_pool, max_pool], axis=3)
      
        attention_map = self.conv(concatenated)
        return inputs * attention_map

spatial_attent = SpatialAttention()
input_tensor = tf.random.normal((1, 7, 7, 2048))  
output_tensor = spatial_attent(input_tensor)
print(output_tensor.shape)  # Output shape should be (1, 7, 7, 2048)

(1, 7, 7, 2048)


In [8]:

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import numpy as np

class CosFaceLayer(tf.keras.layers.Layer):
    def __init__(self, num_classes, s=30.0, m=0.35):
        super(CosFaceLayer, self).__init__()
        self.num_classes = num_classes
        self.s = s
        self.m = m

    def build(self, input_shape):
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[-1], self.num_classes),
                                 initializer='glorot_uniform',
                                 trainable=True)

    def call(self, inputs):
        x = tf.linalg.l2_normalize(inputs, axis=1)

        W = tf.linalg.l2_normalize(self.W, axis=0)

        cos_theta = tf.matmul(x, W)

        return cos_theta * self.s

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_classes)
def cosface_loss(y_true, y_pred):
    m=0.35
    s=30.0
    y_true = tf.cast(tf.reshape(y_true, [-1]), tf.int32)
    cos_theta = tf.reshape(y_pred, [tf.shape(y_true)[0], -1])

    num_classes = tf.shape(cos_theta)[1]
    batch_size = tf.shape(y_true)[0]

    indices = tf.stack([tf.range(batch_size), y_true], axis=1)
    cos_theta_yi = tf.gather_nd(cos_theta, indices)


    cos_theta_yi_m = cos_theta_yi - m

    mask = tf.one_hot(y_true, depth=num_classes)


    cos_theta_yi_m = tf.expand_dims(cos_theta_yi_m, 1)


    updated_logits = cos_theta * (1.0 - mask) + cos_theta_yi_m * mask

    scaled_logits = s * updated_logits

    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=scaled_logits)

    return tf.reduce_mean(losses)

def custom_accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, tf.int64)  
    y_pred_argmax = tf.argmax(y_pred, axis=-1)
    correct = tf.equal(y_true, y_pred_argmax)
    return tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Flatten,Dropout,Reshape,Conv2D,GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import CosineDecayRestarts

input_image = Input(shape=(7, 7, 2048))
x = Conv2D(1024, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4))(input_image)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4))(x)
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)

x = Reshape((512,))(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
identity_features = Dense(512, activation='relu', name='identity_features')(x)
identity_features = BatchNormalization()(identity_features)
identity_features=CosFaceLayer(num_classes=276)(identity_features)
identity_model = Model(inputs=input_image, outputs=identity_features,name='identity_model')

initial_learning_rate = 0.001
first_decay_steps = 1000
lr_schedule = CosineDecayRestarts(
    initial_learning_rate,
    first_decay_steps,
    t_mul=2.0,
    m_mul=0.9,
    alpha=0.1
)

identity_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=1.0),
                       loss=cosface_loss,
                       metrics=[custom_accuracy])

agefeaturesmap = Input(shape=(7, 7, 2048), name='agefeaturesmap')
x = Conv2D(1024, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4))(agefeaturesmap)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4))(x)
x = BatchNormalization()(x)

x = GlobalAveragePooling2D()(x)

x = Reshape((512,))(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu',kernel_regularizer=l2(1e-4))(x)
x = BatchNormalization()(x)

age_estimation_output = Dense(101, activation='softmax', name='age_estimation_output')(x)

age_Estimation_model = Model(inputs=agefeaturesmap, outputs=age_estimation_output,name='age_estimation_model')
age_Estimation_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=1.0),
                             loss='sparse_categorical_crossentropy',
                             metrics=['accuracy'])

identity_model.summary()
age_Estimation_model.summary()

#No use of the combined model architecture as we are only using the identity model from it
combined_input_age = Input(shape=(7, 7, 2048), name='agefeaturesmap')
combined_input_identity = Input(shape=(7,7,2048), name='input_image')

identity_output = identity_model(combined_input_identity)
age_output = age_Estimation_model(combined_input_age)
combined_model = Model(inputs=[combined_input_age, combined_input_identity],
                       outputs=[identity_output, age_output])
combined_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=1.0),
                           loss={
        'identity_model': cosface_loss,
        'age_estimation_model': 'sparse_categorical_crossentropy'
    },
    metrics={
        'identity_model': custom_accuracy,
        'age_estimation_model': 'accuracy'
    }
)
combined_model.build([(7, 7, 2048), (7, 7, 2048)])

combined_model.summary()

Model: "identity_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 7, 7, 2048)]      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 1024)        18875392  
                                                                 
 batch_normalization (Batch  (None, 7, 7, 1024)        4096      
 Normalization)                                                  
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 512)         4719104   
                                                                 
 batch_normalization_1 (Bat  (None, 7, 7, 512)         2048      
 chNormalization)                                                
                                                                 
 global_average_pooling2d (  (None, 512)            

In [10]:
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [11]:
import tensorflow as tf

class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self, lambda_param=1.0):
        super(GradientReversalLayer, self).__init__()
        self.lambda_param = lambda_param

    def call(self, x, training=None):
        if training:
            return tf.reverse_gradient(x, self.lambda_param)
        else:
            return x


In [ ]:
# testing 2
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
img2=Image.open(r'/kaggle/input/ayeshatakiatesting/Screenshot 2024-07-08 193006.png')
img2=img2.convert('RGB')
imgarray2=np.array(img2)
img1=Image.open(r'/kaggle/input/testing2/testsubject2.png')
img1=img1.convert('RGB')
imgarray1=np.array(img1)
layer_norm=tf.keras.layers.LayerNormalization()
layer_resize=tf.keras.layers.Resizing(height=224,width=224)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))
ax1.imshow(img1)
ax1.axis('off') 
ax1.set_title('Image 1')
ax2.imshow(img2)
ax2.axis('off') 
ax2.set_title('Image 2')
ax1.set_aspect('equal')
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()
preprocessed1=layer_resize(layer_norm(imgarray1))
preprocessed2=layer_resize(layer_norm(imgarray2))

prep1=tf.expand_dims(preprocessed1,axis=0)
base1=base_model.predict(prep)

chanoutput1=channel_attent(base1)
spatoutput1=spatial_attent(base1)
attentmask1=spatial_attent(channel_attent(chanoutput1+spatoutput1))
grl = GradientReversalLayer()
identityinput1=grl(attentmask1)*base1

prep2=tf.expand_dims(preprocessed2,axis=0)
base2=base_model.predict(prep2)

chanoutput2=channel_attent(base2)
spatoutput2=spatial_attent(base2)
attentmask2=spatial_attent(channel_attent(chanoutput2+spatoutput2))
grl = GradientReversalLayer()
identityinput2=grl(attentmask2)*base2

identity_model.load_weights(r'/kaggle/working/model_weights2/identity_model_weights25_epoch_10.weights.h5')
identity_features_layer = identity_model.get_layer('identity_features')
identity_features_extractor = Model(inputs=identity_model.input, 
                                    outputs=identity_features_layer.output,
                                    name='identity_features_extractor')
prediction1=(identity_features_extractor.predict(identityinput1))
prediction2=(identity_features_extractor.predict(identityinput2))

def cosine_similarity(a, b):
    a_normalized = tf.nn.l2_normalize(a, axis=-1)
    b_normalized = tf.nn.l2_normalize(b, axis=-1)
    
    return tf.reduce_sum(tf.multiply(a_normalized, b_normalized), axis=-1)
similarity=cosine_similarity(prediction1,prediction2)


from IPython.display import display, HTML
html_string = f'''
<div style="font-size:24px;">
    <p>Percent of Similarity: {similarity.numpy()*100}</p>
</div>
'''
large_text = HTML(html_string)
display(large_text)

In [ ]:
#THE TRAINING IS DONE ON KAGGLE SO HERE IS THE LINK OF KAGGLE 
#https://www.kaggle.com/code/abhiekre/jinx-hopesohopeso


In [ ]:
#TRAINING ARC DOWN BELOW EVERY GENERATOR TARGETS DIFFERENT TENSORS IK ITS NOT THE BEST APPROACH BUT I DIDNT HAVE THAT MUCH RAM SO IF IT WORKS IT WORKS

In [ ]:

img_Tensor5K=np.load(r'/kaggle/input/tensors/img_tensor5k.npy')
label_tensor=np.load(r'/kaggle/input/tensors/labels.npy')
def generator1():
        for i in range(5000):
            img = img_Tensor5K[i]
            label = label_tensor[i]


            inputt = tf.expand_dims(preprocessing(img), axis=0)
  
            feature_map = base_model(inputt, training=False)

            chanoutput = channelattention(feature_map)
            spatialoutput = spatialattention(feature_map)
            attentionmask1 = chanoutput + spatialoutput
            attentionmask2=channelattention(attentionmask1)
            attentionmask=spatialattention(attentionmask2)

            age_input = attentionmask * feature_map
            grl = GradientReversalLayer()
    
            reversed_mask = grl(attentionmask)
    
            identity_input = tf.multiply(reversed_mask, feature_map)

            identity = label[0]
            age = label[1]

            yield (identity_input, age_input), (identity, age)

dataset = tf.data.Dataset.from_generator(
    generator1,
    output_signature=(
        (tf.TensorSpec(shape=(1,7, 7, 2048), dtype=tf.float32),
         tf.TensorSpec(shape=(1, 7, 7, 2048), dtype=tf.float32)),
        (tf.TensorSpec(shape=(), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32))
    )
)

batch_size = 20
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

channelattention = ChannelAttention(2048)
spatialattention = SpatialAttention()

dataset_size = 5000 //20  

train_size = int(0.7 * dataset_size)  
val_size = int(0.15 * dataset_size)  
test_size = dataset_size - train_size - val_size  



train_dataset5k = dataset.take(train_size)
temp_dataset = dataset.skip(train_size)
val_dataset5k = temp_dataset.take(val_size)
test_dataset5k = temp_dataset.skip(val_size)

train_identity_losses = []
train_age_losses = []
val_identity_losses = []
val_age_losses = []

save_dir=r'/kaggle/working/model_weights/'
os.makedirs(save_dir, exist_ok=True)
train_dataset5k = train_dataset5k.shuffle(buffer_size=1000)
identity_model.load_weights(r'/kaggle/working/model_weights/identity_model_weights8_epoch_61.weights.h5')
age_Estimation_model.load_weights(r'/kaggle/working/model_weights/age_model_weights8_epoch_61.weights.h5')
count=55
for epoch in range(5):  
    print(f"Epoch {epoch + 1}/10")
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(train_dataset5k):
     
        print(identity_inputs.shape)
        print(age_inputs.shape)

        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)
        print(identity_inputs.shape)
        print(age_inputs.shape)

        history_identity = identity_model.fit(
            x=identity_inputs, y=identities,
            epochs=1, verbose=0,
            batch_size=20 
        )

        history_age = age_Estimation_model.fit(
            x=age_inputs, y=ages,
            epochs=1, verbose=0,
            batch_size=20  
        )

        print(f"Batch {batch + 1} processed. "
              f"Identity Loss: {history_identity.history['loss'][0]:.4f}, "
              f"Age Loss: {history_age.history['loss'][0]:.4f}")
    val_identity_loss = []
    val_age_loss = []
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(val_dataset5k):
        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)


        val_identity_pred = identity_model.predict(identity_inputs)

        val_identity_loss.append(tf.keras.losses.sparse_categorical_crossentropy(identities, val_identity_pred))

        ages = tf.cast(ages, tf.float32)

        val_age_pred = age_Estimation_model.predict(age_inputs)

        val_age_pred = tf.cast(tf.argmax(val_age_pred, axis=1), tf.float32)

        val_age_loss.append(tf.reduce_mean(tf.abs(ages - val_age_pred)))
        print('validation loss identity:',sum(val_identity_loss) / len(val_identity_loss))
        print('validation loss age:',sum(val_identity_loss) / len(val_identity_loss))
    val_identity_losses.append(sum(val_identity_loss) / len(val_identity_loss))
    val_age_losses.append(sum(val_age_loss) / len(val_age_loss))
    

    if (epoch + 1) % 2 == 1:  
        print(f"Saving weights for epoch {epoch + 1}")
        identity_model.save_weights(os.path.join(save_dir,f'identity_model_weights14_epoch_{count+epoch + 1}.weights.h5'))
        age_Estimation_model.save_weights(os.path.join(save_dir,f'age_model_weights14_epoch_{count+epoch + 1}.weights.h5'))

    count+=1
    tf.keras.backend.clear_session()
epochs = range(1, len(train_identity_losses) + 1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_identity_losses, 'bo-', label='Training Identity Loss')
plt.plot(epochs, val_identity_losses, 'ro-', label='Validation Identity Loss')
plt.title('Identity Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_age_losses, 'bo-', label='Training Age Loss')
plt.plot(epochs, val_age_losses, 'ro-', label='Validation Age Loss')
plt.title('Age Estimation Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

img_Tensor10K=np.load(r'/kaggle/input/tensors/img_tensor10k.npy')

label_tensor=np.load(r'/kaggle/input/tensors/labels.npy')
def generator2():
        gtrcount=5000
        for i in range(0,5000):
            img = img_Tensor10K[i]
            label = label_tensor[gtrcount+i]



            inputt = tf.expand_dims(preprocessing(img), axis=0)
      
            feature_map = base_model(inputt, training=False)

            chanoutput = channelattention(feature_map)
            spatialoutput = spatialattention(feature_map)
            attentionmask1 = chanoutput + spatialoutput
            attentionmask2=channelattention(attentionmask1)
            attentionmask=spatialattention(attentionmask2)

            age_input = attentionmask * feature_map
            grl = GradientReversalLayer()
    
            reversed_mask = grl(attentionmask)
            identity_input = tf.multiply(reversed_mask, feature_map)

            identity = label[0]
            age = label[1]
            gtrcount+=1
            yield (identity_input, age_input), (identity, age)

dataset2 = tf.data.Dataset.from_generator(
    generator2,
    output_signature=(
        (tf.TensorSpec(shape=(1,7, 7, 2048), dtype=tf.float32),
         tf.TensorSpec(shape=(1, 7, 7, 2048), dtype=tf.float32)),
        (tf.TensorSpec(shape=(), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32))
    )
)

batch_size = 30
dataset2 = dataset2.batch(batch_size)

dataset2 = dataset2.prefetch(tf.data.AUTOTUNE)

channelattention = ChannelAttention(2048)
spatialattention = SpatialAttention()

dataset_size = 5000 //30 

train_size = int(0.7 * dataset_size)  
val_size = int(0.15 * dataset_size)  
test_size = dataset_size - train_size - val_size 

train_identity_losses = []
train_age_losses = []
val_identity_losses = []
val_age_losses = []

train_dataset10k = dataset2.take(train_size)
temp_dataset = dataset2.skip(train_size)
val_dataset10k = temp_dataset.take(val_size)
test_datase10k = temp_dataset.skip(val_size)


save_dir=r'/kaggle/working/model_weights'
os.makedirs(save_dir, exist_ok=True)
train_dataset10k = train_dataset10k.shuffle(buffer_size=1000)
identity_model.load_weights(r'/kaggle/working/model_weights/identity_model_weights7_epoch_55.weights.h5',skip_mismatch=True)
age_Estimation_model.load_weights(r'/kaggle/working/model_weights/age_model_weights7_epoch_55.weights.h5',skip_mismatch=True)
count=56
for epoch in range(10):  
    print(f"Epoch {epoch + 1}/10")
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(train_dataset10k):
       
        print(identity_inputs.shape)
        print(age_inputs.shape)

        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)
        print(identity_inputs.shape)
        print(age_inputs.shape)
   
        history_identity = identity_model.fit(
            x=identity_inputs, y=identities,
            epochs=1, verbose=0,
            batch_size=30 
        )

        history_age = age_Estimation_model.fit(
            x=age_inputs, y=ages,
            epochs=1, verbose=0,
            batch_size=30 
        )

        print(f"Batch {batch + 1} processed. "
              f"Identity Loss: {history_identity.history['loss'][0]:.4f}, "
              f"Age Loss: {history_age.history['loss'][0]:.4f}")
    val_identity_loss = []
    val_age_loss = []
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(val_dataset10k):
        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)


        val_identity_pred = identity_model.predict(identity_inputs)

        val_identity_loss.append(tf.keras.losses.sparse_categorical_crossentropy(identities, val_identity_pred))

        ages = tf.cast(ages, tf.float32)

        val_age_pred = age_Estimation_model.predict(age_inputs)

        val_age_pred = tf.cast(tf.argmax(val_age_pred, axis=1), tf.float32)

        val_age_loss.append(tf.reduce_mean(tf.abs(ages - val_age_pred)))
        print("avg val identity loss:",sum(val_identity_loss) / len(val_identity_loss))
        print("avg val age loss",sum(val_age_loss) / len(val_age_loss))
    if(len(val_identity_loss)!=0 and len(val_identity_loss)!=0):
        val_identity_losses.append(sum(val_identity_loss) / len(val_identity_loss))
        val_age_losses.append(sum(val_age_loss) / len(val_age_loss))

    if (epoch + 1) % 2 == 1:  
        print(f"Saving weights for epoch {epoch + 1}")
        identity_model.save_weights(os.path.join(save_dir,f'identity_model_weights8_epoch_{count+epoch + 1}.weights.h5'))
        age_Estimation_model.save_weights(os.path.join(save_dir,f'age_model_weights8_epoch_{count+epoch + 1}.weights.h5'))
    
    count+=1
    
    tf.keras.backend.clear_session()
epochs = range(1, len(train_identity_losses) + 1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_identity_losses, 'bo-', label='Training Identity Loss')
plt.plot(epochs, val_identity_losses, 'ro-', label='Validation Identity Loss')

plt.title('Identity Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_age_losses, 'bo-', label='Training Age Loss')
plt.plot(epochs, val_age_losses, 'ro-', label='Validation Age Loss')
plt.title('Age Estimation Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

img_Tensor15K=np.load(r'/kaggle/input/tensors/img_tensor15k.npy')
label_tensor=np.load(r'/kaggle/input/tensors/labels.npy')
def generator3():
        gtrcount=10000
        for i in range(0,5000):
            img = img_Tensor15K[i]
            label = label_tensor[gtrcount+i]



            inputt = tf.expand_dims(preprocessing(img), axis=0)

            feature_map = base_model(inputt, training=False)

            chanoutput = channelattention(feature_map)
            spatialoutput = spatialattention(feature_map)
            attentionmask1 = chanoutput + spatialoutput
            attentionmask2=channelattention(attentionmask1)
            attentionmask=spatialattention(attentionmask2)

            age_input = attentionmask * feature_map
            grl = GradientReversalLayer()
    
            reversed_mask = grl(attentionmask)
    
            identity_input = tf.multiply(reversed_mask, feature_map)

            identity = label[0]
            age = label[1]
            gtrcount+=1
            yield (identity_input, age_input), (identity, age)

dataset3= tf.data.Dataset.from_generator(
    generator3,
    output_signature=(
        (tf.TensorSpec(shape=(1,7, 7, 2048), dtype=tf.float32),
         tf.TensorSpec(shape=(1, 7, 7, 2048), dtype=tf.float32)),
        (tf.TensorSpec(shape=(), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32))
    )
)
=
batch_size = 30
dataset3 = dataset3.batch(batch_size)

dataset3 = dataset3.prefetch(tf.data.AUTOTUNE)

channelattention = ChannelAttention(2048)
spatialattention = SpatialAttention()

dataset_size = 5000 //30 

train_size = int(0.7 * dataset_size) 
val_size = int(0.15 * dataset_size)  
test_size = dataset_size - train_size - val_size  



train_dataset15k = dataset3.take(train_size)
temp_dataset = dataset3.skip(train_size)
val_dataset15k = temp_dataset.take(val_size)
test_dataset15k = temp_dataset.skip(val_size)

train_identity_losses = []
train_age_losses = []
val_identity_losses = []
val_age_losses = []

save_dir=r'/kaggle/working/model_weights2'
os.makedirs(save_dir, exist_ok=True)
train_dataset15k = train_dataset15k.shuffle(buffer_size=1000)
identity_model.load_weights(r'/kaggle/working/model_weights/identity_model_weights14_epoch_60.weights.h5',skip_mismatch=True)
age_Estimation_model.load_weights(r'/kaggle/working/model_weights/age_model_weights14_epoch_60.weights.h5',skip_mismatch=True)
count=10
for epoch in range(10):  
    print(f"Epoch {epoch + 1}/10")
    epoch_identity_loss = []
    epoch_age_loss = []
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(train_dataset15k):
     

        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)
        print(identity_inputs.shape)
        print(age_inputs.shape)
      
        history_identity = identity_model.fit(
            x=identity_inputs, y=identities,
            epochs=1, verbose=0,
            batch_size=10
        )

        history_age = age_Estimation_model.fit(
            x=age_inputs, y=ages,
            epochs=1, verbose=0,
            batch_size=10  
        )

        print(f"Batch {batch + 1} processed. "
              f"Identity Loss: {history_identity.history['loss'][0]:.4f}, "
              f"Age Loss: {history_age.history['loss'][0]:.4f}")
    val_identity_loss = []
    val_age_loss = []
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(val_dataset15k):
        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)

        val_identity_pred = identity_model.predict(identity_inputs)

        val_identity_loss.append(tf.keras.losses.sparse_categorical_crossentropy(identities, val_identity_pred))

        ages = tf.cast(ages, tf.float32)

        val_age_pred = age_Estimation_model.predict(age_inputs)

        val_age_pred = tf.cast(tf.argmax(val_age_pred, axis=1), tf.float32)

        val_age_loss.append(tf.reduce_mean(tf.abs(ages - val_age_pred)))
        
    val_identity_losses.append(sum(val_identity_loss) / len(val_identity_loss))
    val_age_losses.append(sum(val_age_loss) / len(val_age_loss))
    print("val identiy:",sum(val_identity_loss) / len(val_identity_loss))
    print("val age :",sum(val_age_loss) / len(val_identity_loss))
    if (epoch + 1) % 2 == 1:  
        print(f"Saving weights for epoch {epoch + 1}")
        identity_model.save_weights(os.path.join(save_dir,f'identity_model_weights15_epoch_{count+epoch + 1}.weights.h5'))
        age_Estimation_model.save_weights(os.path.join(save_dir,f'age_model_weights15_epoch_{count+epoch + 1}.weights.h5'))

    count+=1
 
    tf.keras.backend.clear_session()
epochs = range(1, len(train_identity_losses) + 1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_identity_losses, 'bo-', label='Training Identity Loss')
plt.plot(epochs, val_identity_losses, 'ro-', label='Validation Identity Loss')
plt.title('Identity Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_age_losses, 'bo-', label='Training Age Loss')
plt.plot(epochs, val_age_losses, 'ro-', label='Validation Age Loss')
plt.title('Age Estimation Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:


img_Tensor20K = np.load(r'/kaggle/input/tensors/img_tensor20k.npy')
label_tensor = np.load(r'/kaggle/input/tensors/labels.npy')

def generator4():
    gtrcount = 15000
    for i in range(0, 5000):
        img = img_Tensor20K[i]
        label = label_tensor[gtrcount + i]

        inputt = tf.expand_dims(preprocessing(img), axis=0)
      
        feature_map = base_model(inputt, training=False)

        chanoutput = channelattention(feature_map)
        spatialoutput = spatialattention(feature_map)
        attentionmask1 = chanoutput + spatialoutput
        attentionmask2=channelattention(attentionmask1)
        attentionmask=spatialattention(attentionmask2)

        age_input = attentionmask * feature_map
        grl = GradientReversalLayer()
    
        reversed_mask = grl(attentionmask)
    
        identity_input = tf.multiply(reversed_mask, feature_map)
 
        identity = label[0]
        age = label[1]
        gtrcount += 1
        yield (identity_input, age_input), (identity, age)

dataset4 = tf.data.Dataset.from_generator(
    generator4,
    output_signature=(
        (tf.TensorSpec(shape=(1,7, 7, 2048), dtype=tf.float32),
         tf.TensorSpec(shape=(1, 7, 7, 2048), dtype=tf.float32)),
        (tf.TensorSpec(shape=(), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32))
    )
)

batch_size = 10
dataset4 = dataset4.batch(batch_size)

dataset4 = dataset4.prefetch(tf.data.AUTOTUNE)

channelattention = ChannelAttention(2048)
spatialattention = SpatialAttention()
dataset_size = 5000 // 10  

train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)  
test_size = dataset_size - train_size - val_size

train_dataset20k = dataset4.take(train_size)
temp_dataset = dataset4.skip(train_size)
val_dataset20k = temp_dataset.take(val_size)
test_dataset20k = temp_dataset.skip(val_size)


train_identity_losses = []
train_age_losses = []
val_identity_losses = []
val_age_losses = []

save_dir = r'/kaggle/working/model_weights2'
os.makedirs(save_dir, exist_ok=True)
identity_model.load_weights(r'/kaggle/working/model_weights3/identity_model_weights16_epoch_25.weights.h5',skip_mismatch=True)
age_Estimation_model.load_weights(r'/kaggle/working/model_weights3/age_model_weights16_epoch_25.weights.h5',skip_mismatch=True)
train_dataset20k = train_dataset20k.shuffle(buffer_size=1000)
count = 1
for epoch in range(10):
    print(f"Epoch {epoch + 1}/5")
    epoch_identity_loss = []
    epoch_age_loss = []

    train_identity_loss = []
    train_age_loss = []
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(train_dataset20k):
       
        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)

        history_identity = identity_model.fit(
            x=identity_inputs, y=identities,
            epochs=1, verbose=0,
            batch_size=10 
        )

        history_age = age_Estimation_model.fit(
            x=age_inputs, y=ages,
            epochs=1, verbose=0,
            batch_size=10  
        )

        epoch_identity_loss.append(history_identity.history['loss'][0])
        epoch_age_loss.append(history_age.history['loss'][0])
        print(f"Batch {batch + 1} processed. "
              f"Identity Loss: {history_identity.history['loss'][0]:.4f}, "
              f"Age Loss: {history_age.history['loss'][0]:.4f}")
    train_identity_loss.append(history_identity.history['loss'][0])
    train_age_loss.append(history_age.history['loss'][0])

    train_dataset20k = train_dataset20k.shuffle(buffer_size=1000)
 
    val_identity_loss = []
    val_age_loss = []
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(val_dataset20k):
        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)

        val_identity_pred = identity_model.predict(identity_inputs)

        val_identity_loss.append(tf.keras.losses.sparse_categorical_crossentropy(identities, val_identity_pred))

        ages = tf.cast(ages, tf.float32)

        val_age_pred = age_Estimation_model.predict(age_inputs)

        val_age_pred = tf.cast(tf.argmax(val_age_pred, axis=1), tf.float32)

        val_age_loss.append(tf.reduce_mean(tf.abs(ages - val_age_pred)))
    
    val_identity_losses.append(sum(val_identity_loss) / len(val_identity_loss))
    val_age_losses.append(sum(val_age_loss) / len(val_age_loss))
    print("val id loss:",sum(val_identity_loss) / len(val_identity_loss))
    print("val age loss:",sum(val_age_loss) / len(val_age_loss))
    if (epoch + 1) % 2 == 1:  
        print(f"Saving weights for epoch {epoch + 1}")
        identity_model.save_weights(os.path.join(save_dir,f'identity_model_weights25_epoch_{count+epoch + 1}.weights.h5'))
        age_Estimation_model.save_weights(os.path.join(save_dir,f'age_model_weights25_epoch_{count+epoch + 1}.weights.h5'))

    count+=1
   
    print(f"Epoch {epoch+1}")
    print(f"Average Identity Loss: {tf.reduce_mean(val_identity_loss)}")
    print(f"Average Age Loss: {tf.reduce_mean(val_age_loss)}")

In [ ]:

img_Tensor25K=np.load(r'/kaggle/input/tensors/img_tensor25k.npy')
label_tensor=np.load(r'/kaggle/input/tensors/labels.npy')
def generator5():
        gtrcount=20000
        for i in range(0,5000):
            img = img_Tensor25K[i]

            label = label_tensor[gtrcount+i]


            inputt = tf.expand_dims(preprocessing(img), axis=0)
       
            feature_map = base_model(inputt, training=False)

            chanoutput = channelattention(feature_map)
            spatialoutput = spatialattention(feature_map)
            attentionmask1 = chanoutput + spatialoutput
            attentionmask2=channelattention(attentionmask1)
            attentionmask=spatialattention(attentionmask2)

            age_input = attentionmask * feature_map
            grl = GradientReversalLayer()
    
            reversed_mask = grl(attentionmask)
    
            identity_input = tf.multiply(reversed_mask, feature_map)

            identity = label[0]
            age = label[1]
            gtrcount+=1
            yield (identity_input, age_input), (identity, age)

dataset5= tf.data.Dataset.from_generator(
    generator5,
    output_signature=(
        (tf.TensorSpec(shape=(1,7, 7, 2048), dtype=tf.float32),
         tf.TensorSpec(shape=(1, 7, 7, 2048), dtype=tf.float32)),
        (tf.TensorSpec(shape=(), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32))
    )
)
batch_size = 30
dataset5 = dataset5.batch(batch_size)

dataset5 = dataset5.prefetch(tf.data.AUTOTUNE)

channelattention = ChannelAttention(2048)
spatialattention = SpatialAttention()
dataset_size = 5000 //30  

train_size = int(0.7 * dataset_size) 
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

train_identity_losses = []
train_age_losses = []
val_identity_losses = []
val_age_losses = []

train_dataset25k = dataset5.take(train_size)
temp_dataset = dataset5.skip(train_size)
val_dataset25k = temp_dataset.take(val_size)
test_dataset25k = temp_dataset.skip(val_size)



save_dir=r'/kaggle/working/model_weights3'
os.makedirs(save_dir, exist_ok=True)
train_dataset25k = train_dataset25k.shuffle(buffer_size=1000)
identity_model.load_weights(r'/kaggle/working/model_weights3/identity_model_weights16_epoch_21.weights.h5',skip_mismatch=True)
age_Estimation_model.load_weights(r'/kaggle/working/model_weights3/age_model_weights16_epoch_21.weights.h5',skip_mismatch=True)
count=20
val_identity_losses=[]
val_identity_losses=[]
for epoch in range(10):  
    print(f"Epoch {epoch + 1}/10")
    epoch_identity_loss = []
    epoch_age_loss = []
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(train_dataset25k):
   


        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)

        history_identity = identity_model.fit(
            x=identity_inputs, y=identities,
            epochs=1, verbose=0,
            batch_size=30 
        )

        history_age = age_Estimation_model.fit(
            x=age_inputs, y=ages,
            epochs=1, verbose=0,
            batch_size=30 
        )
#
        epoch_identity_loss.append(history_identity.history['loss'][0])
        epoch_age_loss.append(history_age.history['loss'][0])
        print(f"Batch {batch + 1} processed. "
              f"Identity Loss: {history_identity.history['loss'][0]:.4f}, "
              f"Age Loss: {history_age.history['loss'][0]:.4f}")
    train_identity_losses.append(sum(epoch_identity_loss) / len(epoch_identity_loss))
    train_age_losses.append(sum(epoch_age_loss) / len(epoch_age_loss))
    val_age_loss=[]
    val_identity_loss=[]
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(val_dataset25k):
        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)


        val_identity_pred = identity_model.predict(identity_inputs)

        val_identity_loss.append(tf.keras.losses.sparse_categorical_crossentropy(identities, val_identity_pred))

        ages = tf.cast(ages, tf.float32)

        val_age_pred = age_Estimation_model.predict(age_inputs)

        val_age_pred = tf.cast(tf.argmax(val_age_pred, axis=1), tf.float32)

        val_age_loss.append(tf.reduce_mean(tf.abs(ages - val_age_pred)))
        
    val_identity_losses.append(sum(val_identity_loss) / len(val_identity_loss))
    val_age_losses.append(sum(val_age_loss) / len(val_age_loss))
    print("val identiy:",sum(val_identity_loss) / len(val_identity_loss))
    print("val age :",sum(val_age_loss) / len(val_identity_loss))
    if (epoch + 1) % 2 == 1:  
        print(f"Saving weights for epoch {epoch + 1}")
        identity_model.save_weights(os.path.join(save_dir,f'identity_model_weights16_epoch_{count+epoch + 1}.weights.h5'))
        age_Estimation_model.save_weights(os.path.join(save_dir,f'age_model_weights16_epoch_{count+epoch + 1}.weights.h5'))

    count+=1
 
    tf.keras.backend.clear_session()

epochs = range(1, len(train_identity_losses) + 1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_identity_losses, 'bo-', label='Training Identity Loss')
plt.plot(epochs, val_identity_losses, 'ro-', label='Validation Identity Loss')
plt.title('Identity Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_age_losses, 'bo-', label='Training Age Loss')
plt.plot(epochs, val_age_losses, 'ro-', label='Validation Age Loss')
plt.title('Age Estimation Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:



label_tensor=np.load(r'/kaggle/input/tensors/labels.npy')
lable_tensor=label_tensor[25000:30000]
print(len(lable_tensor))

img_Tensor30K=np.load(r'/kaggle/input/tensors/img_tensor30k.npy')
label_tensor=np.load(r'/kaggle/input/tensors/labels.npy')
def generator6():
        for i in range(0,5000):
            img = img_Tensor30K[i]
            label = lable_tensor[i]



            inputt = tf.expand_dims(preprocessing(img), axis=0)
      
            feature_map = base_model(inputt, training=False)

            chanoutput = channelattention(feature_map)
            spatialoutput = spatialattention(feature_map)
            attentionmask1 = chanoutput + spatialoutput
            attentionmask2=channelattention(attentionmask1)
            attentionmask=spatialattention(attentionmask2)


            age_input = attentionmask * feature_map
            grl = GradientReversalLayer()
    
            reversed_mask = grl(attentionmask)
    
            identity_input = tf.multiply(reversed_mask, feature_map)

            identity = label[0]
            age = label[1]
            yield (identity_input, age_input), (identity, age)

dataset6= tf.data.Dataset.from_generator(
    generator6,
    output_signature=(
        (tf.TensorSpec(shape=(1,7, 7, 2048), dtype=tf.float32),
         tf.TensorSpec(shape=(1, 7, 7, 2048), dtype=tf.float32)),
        (tf.TensorSpec(shape=(), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32))
    )
)

batch_size = 30
dataset6 = dataset6.batch(batch_size)


dataset6 = dataset6.prefetch(tf.data.AUTOTUNE)

channelattention = ChannelAttention(2048)
spatialattention = SpatialAttention()

dataset_size = 5000 //30  

train_size = int(0.7 * dataset_size)  
val_size = int(0.15 * dataset_size) 
test_size = dataset_size - train_size - val_size 


train_dataset30k = dataset6.take(train_size)
temp_dataset = dataset6.skip(train_size)
val_dataset30k = temp_dataset.take(val_size)
test_dataset30k = temp_dataset.skip(val_size)

train_identity_losses = []
train_age_losses = []
val_identity_losses = []
val_age_losses = []


save_dir=r'/kaggle/working/model_weights'
os.makedirs(save_dir, exist_ok=True)
train_dataset30k = train_dataset30k.shuffle(buffer_size=1000)
identity_model.load_weights(r'/kaggle/working/model_weights3/age_model_weights16_epoch_29.weights.h5',skip_mismatch=True)
age_Estimation_model.load_weights(r'/kaggle/working/model_weights3/age_model_weights16_epoch_29.weights.h5',skip_mismatch=True)
count=36
for epoch in range(5):  
    print(f"Epoch {epoch + 1}/10")
    epoch_identity_loss = []
    epoch_age_loss = []

    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(train_dataset30k):
      

        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)

        history_identity = identity_model.fit(
            x=identity_inputs, y=identities,
            epochs=1, verbose=0,
            batch_size=20 
        )

        history_age = age_Estimation_model.fit(
            x=age_inputs, y=ages,
            epochs=1, verbose=0,
            batch_size=20 
        )

        epoch_identity_loss.append(history_identity.history['loss'][0])
        epoch_age_loss.append(history_age.history['loss'][0])
        print(f"Batch {batch + 1} processed. "
              f"Identity Loss: {history_identity.history['loss'][0]:.4f}, "
              f"Age Loss: {history_age.history['loss'][0]:.4f}")

    train_identity_losses.append(sum(epoch_identity_loss) / len(epoch_identity_loss))
    train_age_losses.append(sum(epoch_age_loss) / len(epoch_age_loss))
    val_identity_loss = []
    val_age_loss = []
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(val_dataset30k):
        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)


        val_identity_pred = identity_model.predict(identity_inputs)

        val_identity_loss.append(tf.keras.losses.sparse_categorical_crossentropy(identities, val_identity_pred))

        ages = tf.cast(ages, tf.float32)

        val_age_pred = age_Estimation_model.predict(age_inputs)

        val_age_pred = tf.cast(tf.argmax(val_age_pred, axis=1), tf.float32)

        val_age_loss.append(tf.reduce_mean(tf.abs(ages - val_age_pred)))

    val_identity_losses.append(sum(val_identity_loss) / len(val_identity_loss))
    val_age_losses.append(sum(val_age_loss) / len(val_age_loss))
    if (epoch + 1) % 2 == 1: 
        print(f"Saving weights for epoch {epoch + 1}")
        identity_model.save_weights(os.path.join(save_dir,f'identity_model_weights12_epoch_{count+epoch + 1}.weights.h5'))
        age_Estimation_model.save_weights(os.path.join(save_dir,f'age_model_weights12_epoch_{count+epoch + 1}.weights.h5'))
    
    count+=1
   
    tf.keras.backend.clear_session()
epochs = range(1, len(train_identity_losses) + 1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_identity_losses, 'bo-', label='Training Identity Loss')
plt.plot(epochs, val_identity_losses, 'ro-', label='Validation Identity Loss')
plt.title('Identity Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_age_losses, 'bo-', label='Training Age Loss')
plt.plot(epochs, val_age_losses, 'ro-', label='Validation Age Loss')
plt.title('Age Estimation Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:


img_Tensor31K=np.load(r'/kaggle/input/tensors/img_tensor31k.npy')
label_tensor=np.load(r'/kaggle/input/tensors/labels.npy')
labeltensor31=label_tensor[30000:]
def generator7():
        gtrcount=30000
        for i in range(0,1464):
            img = img_Tensor31K[i]
            label = labeltensor31[i]



            inputt = tf.expand_dims(preprocessing(img), axis=0)
      
            feature_map = base_model(inputt, training=False)

            chanoutput = channelattention(feature_map)
            spatialoutput = spatialattention(feature_map)
            attentionmask1 = chanoutput + spatialoutput
            attentionmask2=channelattention(attentionmask1)
            attentionmask=spatialattention(attentionmask2)


            age_input = attentionmask * feature_map
            grl = GradientReversalLayer()
    
            reversed_mask = grl(attentionmask)
    
            identity_input = tf.multiply(reversed_mask, feature_map)

            identity = label[0]
            age = label[1]
            gtrcount+=1
            yield (identity_input, age_input), (identity, age)

dataset7= tf.data.Dataset.from_generator(
    generator7,
    output_signature=(
        (tf.TensorSpec(shape=(1,7, 7, 2048), dtype=tf.float32),
         tf.TensorSpec(shape=(1, 7, 7, 2048), dtype=tf.float32)),
        (tf.TensorSpec(shape=(), dtype=tf.int32),
         tf.TensorSpec(shape=(), dtype=tf.int32))
    )
)

batch_size = 20
dataset7 = dataset7.batch(batch_size)


dataset7 = dataset7.prefetch(tf.data.AUTOTUNE)

channelattention = ChannelAttention(2048)
spatialattention = SpatialAttention()


dataset_size = 1464 //20 

train_size = int(0.7 * dataset_size)  
val_size = int(0.15 * dataset_size) 
test_size = dataset_size - train_size - val_size 
train_dataset31k = dataset7.take(train_size)
temp_dataset = dataset7.skip(train_size)
val_dataset31k = temp_dataset.take(val_size)
test_dataset31k = temp_dataset.skip(val_size)

train_identity_losses = []
train_age_losses = []
val_identity_losses = []
val_age_losses = []

save_dir=r'/kaggle/working/model_weights/'
os.makedirs(save_dir, exist_ok=True)
train_dataset31k = train_dataset31k.shuffle(buffer_size=1000)
count=38
for epoch in range(10):  
    print(f"Epoch {epoch + 1}/10")
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(train_dataset31k):
       
        print(identity_inputs.shape)
        print(age_inputs.shape)

        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)
        print(identity_inputs.shape)
        print(age_inputs.shape)
       
        history_identity = identity_model.fit(
            x=identity_inputs, y=identities,
            epochs=1, verbose=0,
            batch_size=20 
        )

        history_age = age_Estimation_model.fit(
            x=age_inputs, y=ages,
            epochs=1, verbose=0,
            batch_size=20  
        )

        print(f"Batch {batch + 1} processed. "
              f"Identity Loss: {history_identity.history['loss'][0]:.4f}, "
              f"Age Loss: {history_age.history['loss'][0]:.4f}")
    val_identity_loss = []
    val_age_loss = []
    for batch, ((identity_inputs, age_inputs), (identities, ages)) in enumerate(val_dataset31k):
        identity_inputs = tf.squeeze(identity_inputs, axis=1)
        age_inputs = tf.squeeze(age_inputs, axis=1)

        val_identity_pred = identity_model.predict(identity_inputs)

        val_identity_loss.append(tf.keras.losses.sparse_categorical_crossentropy(identities, val_identity_pred))

        ages = tf.cast(ages, tf.float32)

        val_age_pred = age_Estimation_model.predict(age_inputs)

        val_age_pred = tf.cast(tf.argmax(val_age_pred, axis=1), tf.float32)

        val_age_loss.append(tf.reduce_mean(tf.abs(ages - val_age_pred)))
    val_identity_losses.append(sum(val_identity_loss) / len(val_identity_loss))
    val_age_losses.append(sum(val_age_loss) / len(val_age_loss))

    if (epoch + 1) % 2 == 1:  
        print(f"Saving weights for epoch {epoch + 1}")
        identity_model.save_weights(os.path.join(save_dir,f'identity_model_weights7_epoch_{count+epoch + 1}.weights.h5'))
        age_Estimation_model.save_weights(os.path.join(save_dir,f'age_model_weights7_epoch_{count+epoch + 1}.weights.h5'))
   
    count+=1
  
    tf.keras.backend.clear_session()
epochs = range(1, len(train_identity_losses) + 1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_identity_losses, 'bo-', label='Training Identity Loss')
plt.plot(epochs, val_identity_losses, 'ro-', label='Validation Identity Loss')
plt.title('Identity Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_age_losses, 'bo-', label='Training Age Loss')
plt.plot(epochs, val_age_losses, 'ro-', label='Validation Age Loss')
plt.title('Age Estimation Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()